Coursera_Capstone_Project_Week3_Segmenting and Clustering Neighborhoods in Toronto

Let's import some important libraries. 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

We can you pandas.read_html to read our url. Very straightforward isn't it ? ;)

In [2]:
## URL to get the data. 
wikitable = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

## Creating a DataFrame with Pandas.
df_t = pd.read_html(wikitable)[0]

## Trim the column Postal Code.
df_t.rename(columns={'Postal Code' : 'PostalCode'}, inplace = True)

df_t.head(12)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [3]:
## Drop Borough row contqining 'Not assigned'.
df_t = df_t[df_t.Borough != 'Not assigned']
df_t.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
## Combine Neighborhood rows where PostalCode is duplicated.
df_t = df_t.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_t.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
## I prefer to define a function for that purpose.
def neighborhood(row):
    if row['Borough']!='' and row['Neighborhood']=='Not assigned':
        return row['Neighborhood']==row['Borough']
    else:
        return row['Neighborhood']
df_t['Neighborhood'] = df_t.apply(neighborhood, axis = 1)

In [8]:
df_t.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
df_t.shape

(103, 3)

In [6]:
print('The number of rows and columns of the dataframe are : {}'.format(df_t.shape))

The number of rows and columns of the dataframe are : (103, 3)


My assumption is that we are going to populate our data with a Foursquare query,
then perform a cluster segmentation.